# Load libraries and Data

In [1]:
# standard libraries
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 30)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
import math
# plt.figure(figsize=(16,6))  # set the plot size
# plt.ylim(-0.01, 0.04)  # set the axis zoom range
# plt.xlim(-0.01, 0.04)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyhive import presto
import operator

def get_presto_conn():
    return presto.connect(host='presto-alpha-backend.data.houzz.net', port=8086, username='wandajuan')

def presto_query(query, presto_conn, fetchall=True):
    cursor = presto_conn.cursor()
    cursor.execute(query)
    header = list(map(operator.itemgetter(0), cursor.description))
    return header, cursor.fetchall()

conn = get_presto_conn() # establish the connection

In [3]:
!ls

20220504 SUBMISSION_Round 2 - Recommend Pricing Homesquare SKUs.xlsx
re-check pricing copy.ipynb
re-check pricing.ipynb


In [4]:
df = pd.read_excel('20220504 SUBMISSION_Round 2 - Recommend Pricing Homesquare SKUs.xlsx', header=1)
df.head()

,product_code,recommended_price,Comment
0,2336,289,Recommended Price Accepted
1,9522,488,Request Rejected
2,10051,407,Request Rejected
3,24340,322,Request Rejected
4,55187,1254,Request Rejected


In [5]:
df.groupby('Comment')['product_code'].count()

Comment
Recommended Price Accepted    440
Request Rejected              560
Name: product_code, dtype: int64

In [6]:
# refresh crawl list
query = """

with t as (
select house_id, 
		comp_crawled_time, 
		transform(cast(json_parse(json_format(json_extract(comp_details, '$.comp_all'))) AS ARRAY<MAP<VARCHAR, VARCHAR>>), entry->entry['name']) names,
		transform(cast(json_parse(json_format(json_extract(comp_details, '$.comp_all'))) AS ARRAY<MAP<VARCHAR, VARCHAR>>), entry->entry['price']) prices,
		transform(cast(json_parse(json_format(json_extract(comp_details, '$.comp_all'))) AS ARRAY<MAP<VARCHAR, VARCHAR>>), entry->entry['shipping']) ships,
		json_extract(comp_details, '$.comp_all') comp_all
from c2.comp_products_third_party c
where c.comp_crawled_time >= '2022-04-01' -- crawled in last 14days
)
, t2 as (
select 
		*,
		array_position(names, 'HomeSquare') hs_pos -- get the position in the arrays for HS
from t
where contains(names, 'HomeSquare') -- filter for only HS prices
)
, hs_comp as (
select 
		house_id,
		comp_crawled_time, 
		ceil(cast(prices[hs_pos] as double) + cast(ships[hs_pos] as double)) hs_comp_price
from t2
)
, base as (
-- 17,972 hs products w recent min core comp price
-- 16879 
select pmt.house_id, pmt.product_code, pmt.vendor_name, item_gmv_1yr,
		ceil(pmt.final_display_price) final_display_price, 
		ceil(c.min_core_comp_price) min_core_comp_price, 
		hs.hs_comp_price
from shop.product_master_table_v1 pmt
join c2.comp_products_third_party c
on pmt.house_id = c.house_id
left join hs_comp hs
on pmt.house_id = hs.house_id
where vendor_name = 'Homesquare'
and c.min_core_comp_price > 0 -- has comp data
and c.comp_crawled_time >= '2022-04-01' -- crawled in last 14days
)
, raw as (
select 
		hs_comp_price < final_display_price has_lower_hs_comp,
		final_display_price > min_core_comp_price has_lower_min_core_comp,
		min_core_comp_price/final_display_price - 1 >= -0.07 and min_core_comp_price/final_display_price - 1 <0  within_7perc,
		case when final_display_price <= min_core_comp_price then 'already price competitive'
			 when final_display_price <= hs_comp_price then 'already lower or equal to hs_comp_price'
			 when final_display_price > hs_comp_price then 'lower price to match hs_comp_price'
			 when hs_comp_price is null and min_core_comp_price/final_display_price-1 >= -0.07 then 'lower price to match min_core_comp'
			 when hs_comp_price is null and min_core_comp_price/final_display_price-1 < -0.07 then 'remove bcs outside of 7%'
			 else 'other'
			 end as reason,
		case when final_display_price <= min_core_comp_price then NULL
			 when final_display_price <= hs_comp_price then NULL
			 when final_display_price > hs_comp_price then hs_comp_price
			 when hs_comp_price is null and min_core_comp_price/final_display_price-1 >= -0.07 then min_core_comp_price 
			 when hs_comp_price is null and min_core_comp_price/final_display_price-1 < -0.07 then NULL
			 else NULL
			 end as recommended_price,
			 *
			 
--		count(*)
from base
--group by 1, 2, 3, 4
)
select product_code, recommended_price
		, house_id, final_display_price
from raw
where recommended_price is not null
order by item_gmv_1yr desc
"""
# print(query)

crawl = pd.read_sql(query, conn)
crawl

,product_code,recommended_price,house_id,final_display_price
0,1644098,294.0,40018136,310.0
1,1998323,272.0,182577871,273.0
2,1767030,590.0,127548754,596.0
3,1648998,446.0,77625429,453.0
4,2025836,2106.0,165582411,2191.0
...,...,...,...,...
12991,1567808,214.0,102466921,221.0
12992,1855129,310.0,125976999,322.0
12993,2071873,155.0,174250047,191.0
12994,2152081,173.0,168009222,180.0


In [7]:
# refresh prefer pricing

query = """
with hs as (
	select house_id, vendor_listing_id, preferred_vendor_listing_id, 
			vendor_name, product_code, ceil(final_display_price) final_display_price,  
			if(availability=1 and quantity >0, 0, 1) as OOS
	from shop.vl_pupil 
	where vendor_name = 'Homesquare' and all_status_valid = true
	and vendor_listing_id != preferred_vendor_listing_id
)
, raw as (
	select hs.*, vl.vendor_name pref_vendor_name, ceil(vl.final_display_price) pref_final_display_price, vl.trade_display_price pref_trade_display_price,
			case when OOS = 1 then 'OOS'
				 when vl.trade_display_price > 0 then 'trade'
				 when hs.final_display_price >= vl.final_display_price * (1-0.015) then 'consumer'
				 else 'other'
				 end as reason,
			case when OOS = 1 then NULL
				 when vl.trade_display_price > 0 then vl.trade_display_price 
				 when hs.final_display_price >= vl.final_display_price * (1-0.015) then floor(vl.final_display_price * (1-0.015) -1)
				 else NULL
				 end as recommended_price		
	from shop.vl_pupil vl
	inner join hs
	on vl.vendor_listing_id = hs.preferred_vendor_listing_id
	where vl.vendor_name != 'Homesquare' and vl.all_status_valid = true
	and vl.vendor_listing_id = vl.preferred_vendor_listing_id
)
-- 7536 to suggest price decrease within 7%
select 
		raw.product_code, recommended_price
		, raw.vendor_listing_id, raw.final_display_price 
from raw 
left join shop.product_master_table_v1 pmt
on raw.house_id = pmt.house_id
where 1.00*recommended_price/raw.final_display_price-1 < 0 
and 1.00*recommended_price/raw.final_display_price-1 >= -0.07
order by item_gmv_1yr desc

"""

prefer = pd.read_sql(query, conn)
prefer

,product_code,recommended_price,vendor_listing_id,final_display_price
0,1469133,135.0,52189188,144.0
1,553996,906.0,63750730,959.0
2,2158367,154.0,118391216,161.0
3,2251082,804.0,119557429,818.0
4,2163140,168.0,117570241,179.0
...,...,...,...,...
7218,2193139,125.0,120336807,134.0
7219,2116752,3051.0,115855932,3223.0
7220,2377183,127.0,121212139,136.0
7221,2305999,181.0,122717088,193.0


In [8]:
new_price = pd.concat([crawl, prefer])
new_price['type'] = new_price['house_id'].apply(lambda x: 'prefer' if pd.isna(x)  else 'crawl')
new_price

,product_code,recommended_price,house_id,final_display_price,vendor_listing_id,type
0,1644098,294.0,40018136.0,310.0,NaN,crawl
1,1998323,272.0,182577871.0,273.0,NaN,crawl
2,1767030,590.0,127548754.0,596.0,NaN,crawl
3,1648998,446.0,77625429.0,453.0,NaN,crawl
4,2025836,2106.0,165582411.0,2191.0,NaN,crawl
...,...,...,...,...,...,...
7218,2193139,125.0,NaN,134.0,120336807.0,prefer
7219,2116752,3051.0,NaN,3223.0,115855932.0,prefer
7220,2377183,127.0,NaN,136.0,121212139.0,prefer
7221,2305999,181.0,NaN,193.0,122717088.0,prefer


In [38]:
df_accepted = df.loc[df['Comment']=='Recommended Price Accepted']
df_accepted['product_code'] = df_accepted['product_code'].astype(str)
df_new_price = df_accepted.merge(new_price, on='product_code', how='left')

In [39]:
df_new_price.to_clipboard()

In [40]:
(df_new_price['recommended_price_x']!=df_new_price['recommended_price_y']).sum()

267

In [28]:
df_new_price.loc[df_new_price['recommended_price_x']!=df_new_price['recommended_price_y']]

,product_code,recommended_price_x,Comment,recommended_price_y,house_id,final_display_price,vendor_listing_id,type
4,367165,3977,Recommended Price Accepted,4371.0,NaN,4617.0,107488783.0,prefer
6,430417,2253,Recommended Price Accepted,2480.0,NaN,2620.0,107964105.0,prefer
8,436143,275,Recommended Price Accepted,270.0,62180112.0,279.0,NaN,crawl
9,440155,1367,Recommended Price Accepted,1505.0,NaN,1591.0,107489108.0,prefer
15,448660,415,Recommended Price Accepted,361.0,66554327.0,365.0,NaN,crawl
...,...,...,...,...,...,...,...,...
279,2209805,234,Recommended Price Accepted,233.0,NaN,236.0,118670113.0,prefer
281,2212006,3977,Recommended Price Accepted,3784.0,NaN,3918.0,119210586.0,prefer
284,2225299,419,Recommended Price Accepted,384.0,NaN,408.0,118984108.0,prefer
285,2229092,2627,Recommended Price Accepted,2884.0,NaN,3047.0,119121758.0,prefer


In [35]:
new_price.loc[new_price['product_code'].isin(['1819760', '1702353'])]

,product_code,recommended_price,house_id,final_display_price,vendor_listing_id,type
